In [2]:
import string
import numpy as np
from PIL import Image
import os
import pickle
from pickle import dump, load
import numpy as np
from keras.utils import plot_model
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.utils import img_to_array , load_img
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout


ModuleNotFoundError: No module named 'tensorflow'

In [6]:
def load_document(filename):
  file = open(filename,'r')
  data = file.read()
  file.close()
  return data

In [10]:
def load_image_captions(filename):
  file = load_document(filename)
  img_captions = file.split('\n')
  del img_captions[0]
  descriptions = {}
  rearranged = []

  for img_caption in img_captions:
    index = img_caption.find(',')
    img_caption = img_caption[:index] + "\t" + img_caption[index+1:] 
    rearranged.append(img_caption)
  img_captions = rearranged
  
  for img_caption in img_captions:
    image , caption = img_caption.split('\t')
    if image not in descriptions:
      descriptions[image] = [caption]
    else:
      descriptions[image].append(caption)
  return descriptions

In [148]:
def cleaning_captions(captions):
  table = str.maketrans('','',string.punctuation)
  for image,img_captions in captions.items():
    for index, image_caption in enumerate(img_captions):
      image_caption.replace("-","")
      caption_words = image_caption.split()
      caption_words = [word.lower() for word in caption_words]
      caption_words = [word.translate(table) for word in caption_words]
      caption_words = [word for word in caption_words if len(word)>1]
      caption_words = [word for word in caption_words if word.isalpha()]
      image_caption = ' '.join(caption_words)
      captions[image][index] = image_caption
  return captions

In [149]:
def vocabulary(image_captions):
  vocabulary = set()
  for image in image_captions.keys():
    [vocabulary.update(caption.split()) for caption in image_captions[image]]
  return vocabulary

In [150]:
def save_to_file(image_captions,filename):
  image_captions_list = list()
  for image,captions in image_captions.items():
    for caption in captions:
      image_captions_list.append(image + '\t' + caption)
  data = "\n".join(image_captions_list)
  file = open(filename,'w')
  file.write(data)
  file.close()

In [176]:
filename = "C:/Users/SAIF UR REHMAN/Desktop/Flickr8K/captions.txt"
image_caption = load_image_captions(filename)
image_captions = cleaning_captions(image_caption)
language_vocabulary = vocabulary(image_captions)
save_to_file(image_captions,'image_captions.txt')

In [177]:
def extract_features(file_location):
  model = Xception(include_top = False,pooling = 'avg')
  features = {}
  for image in os.listdir(file_location):
    img = file_location + '/' + image 
    img = Image.open(img)
    img = img.resize((299,299))
    img = np.expand_dims(img, axis=0)
    img = img/127.5
    img = img - 1.0
    
    feature = model.predict(img)
    features[image] = feature
  return features
      

file_location = "C:/Users/SAIF UR REHMAN/Desktop/Flickr8K/Images"
features = extract_features(file_location)

1/1 [==============================] - 0s 401ms/step


KeyboardInterrupt: 

In [178]:
dump(features,open("Imagefeatures.p","wb"))
len(features)

8091

In [179]:
features

{'1000268201_693b08cb0e.jpg': array([[0.47339708, 0.01732647, 0.07333975, ..., 0.08559047, 0.02102302,
         0.23766533]], dtype=float32),
 '1001773457_577c3a7d70.jpg': array([[0.0015821 , 0.11113498, 0.00037395, ..., 0.26503608, 0.35279873,
         0.0587164 ]], dtype=float32),
 '1002674143_1b742ab4b8.jpg': array([[0.        , 0.02488978, 0.01554056, ..., 0.        , 0.        ,
         0.10192633]], dtype=float32),
 '1003163366_44323f5815.jpg': array([[0.14568894, 0.00272413, 0.2777654 , ..., 0.17018272, 0.11957324,
         0.0941408 ]], dtype=float32),
 '1007129816_e794419615.jpg': array([[0.        , 0.12443952, 0.73916197, ..., 0.00390435, 0.00997151,
         0.50172335]], dtype=float32),
 '1007320043_627395c3d8.jpg': array([[0.04136691, 0.        , 0.01274476, ..., 0.00944688, 0.64201665,
         0.04792134]], dtype=float32),
 '1009434119_febe49276a.jpg': array([[0.        , 0.        , 0.02624731, ..., 0.3052857 , 0.23091805,
         0.14191046]], dtype=float32),
 '1012

In [190]:
def load_imgNames(description):
    return list(description.keys())

In [191]:
def load_clean_captions(filename,imgNames):
    imgCaptions = load_document(filename)
    imgCaptions = imgCaptions.split('\n')
    descriptions = {}
    for imgCaption in imgCaptions:
        words = imgCaption.split()
        if len(words) < 1:
            continue
        
        imgName = words[0]
        imgCap = words[1:]
        
        if imgName in imgNames:
            if imgName not in descriptions:
                descriptions[imgName] = []
            
            imgCap = "<start> " + " ".join(imgCap) + " <end>"
            descriptions[imgName].append(imgCap)
    
    return descriptions
    

In [192]:
# def load_imgFeatures(imgName):
#     all_features = load(open("Imagefeatures.p","rb"))
#     features = {k:all_features[k] for k in imgName}
#     return features

In [220]:
clean_ImgCaptions = load_clean_captions('C:/Users/SAIF UR REHMAN/Desktop/Flickr8K/image_captions.txt',load_imgNames(image_captions)[:-1])
imgNames = load_imgNames(clean_ImgCaptions)
# load_imgFeatures(imgNames)

In [221]:
def list_captions(imgCaptions):
    imgNames = imgCaptions.keys()
    captions_list = []
    for imgName in imgNames:
        [captions_list.append(imgCaption) for imgCaption in imgCaptions[imgName]]
    return captions_list

In [222]:
def create_Tokenizer(imgCaptions):
    captions_list = list_captions(imgCaptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(captions_list)
    return tokenizer

In [223]:
tokenizer = create_Tokenizer(clean_ImgCaptions)
dump(tokenizer,open('tokens.p','wb'))
vocabulary_size = len(tokenizer.word_index) + 1

In [224]:
def max_length(imgCaptions):
    captions_list = list_captions(imgCaptions)
    return max(len(imgCaption.split()) for imgCaption in captions_list)

maxlength = max_length(image_caption)
maxlength

32

In [242]:
vocabulary_size

8764

In [230]:
def create_sequences(tokenizer, max_length, img_captions, feature):
    X1, X2, y = list(), list(), list()
    for img_caption in img_captions:
        
        seq = tokenizer.texts_to_sequences([img_caption])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocabulary_size)[0]
            
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

def data_generator(imgCaptions, features, tokenizer, max_length):
    while 1:
        for key, img_captions in imgCaptions.items():
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, img_captions, feature)
            yield [[input_image, input_sequence], output_word]


In [215]:
[a,b],c = next(data_generator(clean_ImgCaptions, features, tokenizer, maxlength))
a.shape, b.shape, c.shape

((47, 2048), (47, 32), (47, 8764))

In [252]:
def define_model(vocabulary_size, max_length):
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocabulary_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    decoder1 = concatenate([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocabulary_size, activation='softmax')(decoder2)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [256]:
model = define_model(vocabulary_size, maxlength)
epochs = 10
steps = len(clean_ImgCaptions)
for i in range(epochs):
    generator = data_generator(clean_ImgCaptions, features, tokenizer, maxlength)
    model.fit(generator, epochs=1, steps_per_epoch= steps, verbose=1)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 32)]         0           []                               
                                                                                                  
 input_19 (InputLayer)          [(None, 2048)]       0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 32, 256)      2243584     ['input_20[0][0]']               
                                                                                                  
 dropout_13 (Dropout)           (None, 2048)         0           ['input_19[0][0]']               
                                                                                            

KeyboardInterrupt: 

In [17]:
import tensorflow as tf
tf.test.is_gpu_available()

AttributeError: type object 'h5py.h5.H5PYConfig' has no attribute '__reduce_cython__'